In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import AlexNet_Weights
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.447), (0.247, 0.243, 0.262))])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

cuda


In [3]:
AlexNet = models.alexnet(weights=AlexNet_Weights.DEFAULT)
AlexNet.classifier[6] = nn.Linear(4096, 10)
AlexNet.to(device)
print(AlexNet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
# optimizer
optimizer = optim.SGD(AlexNet.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [5]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [6]:
# Validation Function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in tqdm(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)
    
    # Calculate metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1

In [7]:
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for  data, target in tqdm(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [10]:
import torch
import torch.nn.functional as F

def weighted_vote_prediction(model, images, transformations, weights, device, num_classes=10):
    """
      weighted_preds: Tensor of shape [batch_size] with the weighted vote result.
      accepted_mask: Boolean tensor of shape [batch_size] indicating accepted samples.
    """
    model.eval()

    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape: [batch_size]

    # Initialize weighted vote sum using one-hot encoding of original predictions and its weight.
    weighted_sum = F.one_hot(original_preds, num_classes=num_classes).float() * weights[0]

    # 2) For each metamorphic transformation, get predictions and add weighted one-hot votes.
    for i, tf in enumerate(transformations):
        # Apply transformation on CPU and then move to device.
        x_tf = tf(images.cpu()).to(device)
        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)
        # Convert predictions to one-hot and multiply by weight.
        weighted_sum += F.one_hot(preds_tf, num_classes=num_classes).float() * weights[i+1]

    # 3) Final weighted vote: for each sample, take the argmax of the weighted sum.
    weighted_preds = weighted_sum.argmax(dim=1)  # shape: [batch_size]

    # 4) Determine acceptance: accept if weighted vote equals the original prediction.
    accepted_mask = (weighted_preds == original_preds)

    return weighted_preds, accepted_mask


In [11]:
from tqdm import tqdm

def test_with_weighted_vote(model, loader, transformations, weights, device):
    """
    Evaluates the model on the test set 'loader' using weighted voting.
    For each batch:
      - Computes weighted vote predictions (using original + transformed outputs).
      - Accepts a sample only if the weighted vote equals the original prediction.
    
    Prints per-batch information and returns:
      - The accuracy among accepted samples.
      - The overall acceptance rate.
    """
    model.eval()
    total_samples = 0
    total_accepted = 0
    correct_accepted = 0

    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)

        # Get weighted vote predictions and acceptance mask for the batch.
        weighted_preds, accepted_mask = weighted_vote_prediction(model, images, transformations, weights, device)

        batch_size = labels.size(0)
        total_samples += batch_size
        batch_accepted = accepted_mask.sum().item()
        total_accepted += batch_accepted

        # Compute correct predictions among accepted samples.
        if batch_accepted > 0:
            batch_correct = (weighted_preds[accepted_mask] == labels[accepted_mask]).sum().item()
            correct_accepted += batch_correct

        print(f"Batch {i} | Accepted: {batch_accepted}/{batch_size}")

    overall_acceptance_rate = 100.0 * total_accepted / total_samples
    accepted_accuracy = 100.0 * correct_accepted / total_accepted if total_accepted > 0 else 0.0

    print(f"\nOverall Acceptance Rate: {overall_acceptance_rate:.2f}%")
    print(f"Accuracy among Accepted Samples: {accepted_accuracy:.2f}%")
    
    return accepted_accuracy, overall_acceptance_rate


In [13]:
weights = [0.5, 0.25, 0.25, 0.25]

acc_weighted, acceptance_rate = test_with_weighted_vote(AlexNet, testloader, transformations, weights, device)
print(f"Weighted Vote Accuracy among accepted samples: {acc_weighted:.2f}%")
print(f"Overall Acceptance Rate: {acceptance_rate:.2f}%")

  1%|▌                                                                                     | 2/313 [00:00<01:44,  2.98it/s]

Batch 0 | Accepted: 32/32
Batch 1 | Accepted: 30/32


  1%|█                                                                                     | 4/313 [00:01<01:12,  4.26it/s]

Batch 2 | Accepted: 30/32
Batch 3 | Accepted: 30/32


  2%|█▋                                                                                    | 6/313 [00:01<00:54,  5.65it/s]

...

Batch 308 | Accepted: 30/32
Batch 309 | Accepted: 30/32


100%|███████████████████████████████████████████████████████████████████████████████████▋| 312/313 [00:36<00:00,  8.22it/s]

Batch 310 | Accepted: 31/32
Batch 311 | Accepted: 29/32


100%|████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:36<00:00,  8.66it/s]

Batch 312 | Accepted: 15/16

Overall Acceptance Rate: 93.89%
Accuracy among Accepted Samples: 9.85%
Weighted Vote Accuracy among accepted samples: 9.85%
Overall Acceptance Rate: 93.89%
